In [14]:
import numpy as np
import pickle
import os

seed = 2023

In [15]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'cuda'

In [16]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [17]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [18]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [19]:
%run ../load_anno_mi.py

In [20]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-17 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-17 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-17 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-17 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-17 00:00:34


In [21]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(9699, 384)

# Baseline: Fine-tune BERT for classification

In [22]:
num_epochs = 10
seeds = [0, 1, 12, 123, 1234]
validation_metric = "f1"

In [23]:
bert_classifier = fine_tune_transformer_average_seed(num_epochs=num_epochs,
                                                     pretrained_model_name="bert-base-uncased",
                                                     df=anno_mi,
                                                     feature_name="utterance_text",
                                                     label_column="client_talk_type",
                                                     seeds=seeds,
                                                     path_indices=client_index,
                                                     k_fold=True,
                                                     validation_metric=validation_metric,
                                                     results_output=f"{output_dir}/bert_classifier.csv",
                                                     device=device,
                                                     verbose=False)

TypeError: fine_tune_transformer_average_seed() got an unexpected keyword argument 'path_indices'

In [ ]:
bert_classifier

In [ ]:
bert_classifier["f1"].mean()

In [ ]:
bert_classifier["precision"].mean()

In [ ]:
bert_classifier["recall"].mean()

In [ ]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)